In [136]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import layers, regularizers
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [146]:
# Define scaling approach:
# 1. log10 transformation
# 2. standard scaling
# 3. clip to maximum range
standard_scaler = StandardScaler()
min_max_scaler = MinMaxScaler(feature_range=(0,1)) 
cycif_scale_func = lambda d: min_max_scaler.fit_transform( standard_scaler.fit_transform(np.log10(d)).clip(min=-10, max=10) )

def load_cycif_sample(file_name, scale_fn, patient_id, bx_name):
    sample_df = pd.read_csv(file_name, header=0)
    sample_df['patient_id'] = patient_id
    sample_df['bx'] = bx_name
    sample_df['scaled'] = 'N'
    sample_df_scaled = sample_df.copy()
    marker_cols = sample_df.filter(regex="nucleiMasks$").filter(regex="^(?!(DAPI|AF))").columns
    sample_df_scaled[marker_cols] = scale_fn(sample_df[marker_cols])
    sample_df_scaled['scaled'] = 'Y'
    sample_df_all = pd.concat([sample_df, sample_df_scaled])
    return sample_df_all

In [147]:
# Load the dataset.
p9_2_bx1_filename = "./9_2_bx1_HMS_cycif_tumor_panel_BEMS0000342251.csv"
p9_2_bx1_df = load_cycif_sample(p9_2_bx1_filename, cycif_scale_func, "9-2", 1)
p9_2_bx1_df.head()

,CellID,DAPI_1_cellMasks,AF488_cellMasks,AF555_cellMasks,AF647_cellMasks,DAPI_2_cellMasks,ERK1_1_cellMasks,ERK1_2_cellMasks,RAD51_cellMasks,DAPI_3_cellMasks,Cyc_D1_cellMasks,Vimentin_cellMasks,aSMA_cellMasks,DAPI_4_cellMasks,E_cadherin_cellMasks,ER_alpha_cellMasks,PR_cellMasks,DAPI_5_cellMasks,EGFR_cellMasks,Rb_cellMasks,CD45_cellMasks,DAPI_6_cellMasks,Ki67_cellMasks,CK19_cellMasks,p21_cellMasks,DAPI_7_cellMasks,CK14_cellMasks,AR_cellMasks,cPARP_cellMasks,DAPI_8_cellMasks,CK17_cellMasks,CK7_cellMasks,HER2_cellMasks,X_centroid,Y_centroid,column_centroid,row_centroid,Area,MajorAxisLength,MinorAxisLength,Eccentricity,Solidity,Extent,Orientation,DAPI_1_nucleiMasks,AF488_nucleiMasks,AF555_nucleiMasks,AF647_nucleiMasks,DAPI_2_nucleiMasks,ERK1_1_nucleiMasks,ERK1_2_nucleiMasks,RAD51_nucleiMasks,DAPI_3_nucleiMasks,Cyc_D1_nucleiMasks,Vimentin_nucleiMasks,aSMA_nucleiMasks,DAPI_4_nucleiMasks,E_cadherin_nucleiMasks,ER_alpha_nucleiMasks,PR_nucleiMasks,DAPI_5_nucleiMasks,EGFR_nucleiMasks,Rb_nucleiMasks,CD45_nucleiMasks,DAPI_6_nucleiMasks,Ki67_nucleiMasks,CK19_nucleiMasks,p21_nucleiMasks,DAPI_7_nucleiMasks,CK14_nucleiMasks,AR_nucleiMasks,cPARP_nucleiMasks,DAPI_8_nucleiMasks,CK17_nucleiMasks,CK7_nucleiMasks,HER2_nucleiMasks,patient_id,bx,scaled
0,1,18399.201646,1938.308642,951.753086,1536.115226,6840.255144,1285.267490,883.053498,1218.925926,14302.860082,1766.222222,280.168724,328.765432,9072.860082,1391.753086,358.102881,209.806584,73.526749,1760.004115,493.156379,1883.617284,572.576132,1463.831276,16.485597,514.773663,94.288066,2164.238683,379.246914,2489.053498,198.650206,2298.633745,141.518519,2568.308642,10001.341564,668.242798,10001.341564,668.242798,243,23.627398,13.652582,0.816158,0.956693,0.694286,-1.476725,18399.201646,1938.308642,951.753086,1536.115226,6840.255144,1285.267490,883.053498,1218.925926,14302.860082,1766.222222,280.168724,328.765432,9072.860082,1391.753086,358.102881,209.806584,73.526749,1760.004115,493.156379,1883.617284,572.576132,1463.831276,16.485597,514.773663,94.288066,2164.238683,379.246914,2489.053498,198.650206,2298.633745,141.518519,2568.308642,9-2,1,N
1,2,13872.480000,1933.280000,895.360000,1496.160000,6647.280000,1241.400000,864.480000,1408.320000,14804.480000,1709.480000,312.600000,327.840000,7433.440000,1342.560000,340.280000,195.160000,69.760000,1778.800000,498.720000,1894.000000,569.400000,1477.040000,17.640000,523.680000,95.720000,2175.960000,381.560000,2500.080000,192.200000,2289.400000,141.720000,2591.960000,10017.880000,671.240000,10017.880000,671.240000,25,8.451853,4.046502,0.877940,0.833333,0.625000,1.383738,13872.480000,1933.280000,895.360000,1496.160000,6647.280000,1241.400000,864.480000,1408.320000,14804.480000,1709.480000,312.600000,327.840000,7433.440000,1342.560000,340.280000,195.160000,69.760000,1778.800000,498.720000,1894.000000,569.400000,1477.040000,17.640000,523.680000,95.720000,2175.960000,381.560000,2500.080000,192.200000,2289.400000,141.720000,2591.960000,9-2,1,N
2,3,10940.505263,11353.252632,788.294737,4166.000000,119.452632,898.242105,379.168421,823.926316,143.189474,1228.400000,149.863158,299.642105,139.421053,1299.115789,792.568421,275.410526,230.221053,1875.473684,608.410526,1953.800000,235.189474,1490.694737,123.452632,532.494737,151.168421,2197.578947,385.263158,2720.484211,198.568421,2344.273684,146.221053,2580.915789,2824.242105,1353.473684,2824.242105,1353.473684,95,14.535826,8.495993,0.811404,0.950000,0.811966,-1.299994,10940.505263,11353.252632,788.294737,4166.000000,119.452632,898.242105,379.168421,823.926316,143.189474,1228.400000,149.863158,299.642105,139.421053,1299.115789,792.568421,275.410526,230.221053,1875.473684,608.410526,1953.800000,235.189474,1490.694737,123.452632,532.494737,151.168421,2197.578947,385.263158,2720.484211,198.568421,2344.273684,146.221053,2580.915789,9-2,1,N
3,4,10590.968254,8699.428571,714.428571,2570.920635,117.904762,926.841270,395.603175,832.333333,130.253968,1257.730159,138.920635,315.920635,150.984127,1270.555556,439.952381,238.825397,245.9

In [148]:
# Create the VAE using the approach outlined at https://keras.io/examples/generative/vae/

# Create sampling layer.
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a cell."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [149]:
# Build the encoder.

marker_cols = p9_2_bx1_df.filter(regex="nucleiMasks$").filter(regex="^(?!(DAPI|AF))").columns
p9_2_bx1_df = p9_2_bx1_df[p9_2_bx1_df.scaled == 'Y']
markers_df = p9_2_bx1_df[marker_cols].reset_index(drop=True)

latent_dim = 5
inputs_dim = markers_df.shape[1]
r = regularizers.l1(10e-5)
# activation = "relu"

encoder_inputs = keras.Input(shape=(inputs_dim,))
x = layers.Dense(inputs_dim)(encoder_inputs)
x = layers.Dense(inputs_dim / 2)(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLayer)           [(None, 21)]         0                                            
__________________________________________________________________________________________________
dense_51 (Dense)                (None, 21)           462         input_26[0][0]                   
__________________________________________________________________________________________________
dense_52 (Dense)                (None, 10)           220         dense_51[0][0]                   
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 5)            55          dense_52[0][0]                   
____________________________________________________________________________________________

In [150]:
# Build the decoder.
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(inputs_dim / 2)(latent_inputs)
decoder_outputs = layers.Dense(inputs_dim)(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        [(None, 5)]               0         
_________________________________________________________________
dense_53 (Dense)             (None, 10)                60        
_________________________________________________________________
dense_54 (Dense)             (None, 21)                231       
Total params: 291
Trainable params: 291
Non-trainable params: 0
_________________________________________________________________


In [151]:
# Define VAE as a Model with a custom train_step
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss_fn = keras.losses.MeanSquaredError()
            reconstruction_loss = reconstruction_loss_fn(data, reconstruction)
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [152]:
# Create the VAR, compile, and run.
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
history = vae.fit(markers_df.to_numpy(), epochs=100, batch_size=128)

Epoch 1/100
216/216 [==============================] - 1s 927us/step - loss: 0.5617 - reconstruction_loss: 0.2067 - kl_loss: 0.0937
Epoch 2/100
216/216 [==============================] - 0s 930us/step - loss: 0.0675 - reconstruction_loss: 0.0426 - kl_loss: 0.0114
Epoch 3/100
216/216 [==============================] - 0s 926us/step - loss: 0.0361 - reconstruction_loss: 0.0290 - kl_loss: 0.0052
Epoch 4/100
216/216 [==============================] - 0s 1ms/step - loss: 0.0317 - reconstruction_loss: 0.0281 - kl_loss: 0.0029
Epoch 5/100
216/216 [==============================] - 0s 959us/step - loss: 0.0298 - reconstruction_loss: 0.0279 - kl_loss: 0.0017
Epoch 6/100
216/216 [==============================] - 0s 947us/step - loss: 0.0291 - reconstruction_loss: 0.0279 - kl_loss: 0.0010
Epoch 7/100
216/216 [==============================] - 0s 1ms/step - loss: 0.0286 - reconstruction_loss: 0.0278 - kl_loss: 6.7897e-04
Epoch 8/100
216/216 [==============================] - 0s 977us/step - loss:

In [153]:

# Plot performance.
loss_df = pd.DataFrame()
for key in history.history:
    loss_df[key] = history.history[key]
fig = px.line(loss_df)
fig.update_xaxes(title=dict(text='step'))
fig.show()